In [8]:
import random
import string
import time


class Agency:
    """
    A class to manage the generation and validation of one-time codes (OTCs).
    """
    otc_store = {}

    @classmethod
    def generate_otc(cls, user_id):
        """
        Generate a one-time code (OTC) for a specific user.
        OTCs are valid for 5 minutes.
        """
        otc = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))
        expiration_time = time.time() + 300  # OTC valid for 300 seconds (5 minutes)
        cls.otc_store[user_id] = {'otc': otc, 'expires_at': expiration_time}
        return otc
    
    @classmethod
    def validate_otc(cls, user_id, otc):
        """
        Validate a one-time code (OTC) for a specific user.
        """
        if user_id not in cls.otc_store:
            return False

        stored_otc = cls.otc_store[user_id]
        if time.time() > stored_otc['expires_at']:
            # OTC expired
            del cls.otc_store[user_id]
            return False

        if stored_otc['otc'] == otc:
            # Valid OTC
            del cls.otc_store[user_id]  # OTC can only be used once
            return True

        return False


In [9]:



from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.fernet import Fernet
import bcrypt
import os
import json
import pickle  # For binary storage of encrypted tasks


class User:
    # A class-level dictionary to store registered agents for uniqueness checks
    registered_agents = {}

    def __init__(self, agent_id, username, password):
        """
        Initialize a User object with an ID, username, and hashed password.
        """
        self.agent_id = agent_id
        self.username = username
        self.hashed_password = self._hash_password(password)
        self._encryption_key = self._derive_key(password)
        self.encrypted_tasks = []
    
    def _hash_password(self, password):
        """
        Hash the password using bcrypt.
        """
        return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())
    
    def verify_password(self, password):
        """
        Verify the given password against the hashed password.
        """
        return bcrypt.checkpw(password.encode('utf-8'), self.hashed_password)
    
    def _derive_key(self, password):
        """
        Derive a strong encryption key from the user's password using PBKDF2.
        """
        # Use PBKDF2 to derive a key (use a salt for better security)
        salt = os.urandom(16)  # 16-byte random salt
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,  # AES-256 requires a 256-bit key (32 bytes)
            salt=salt,
            iterations=100000,
            backend=default_backend()
        )
        return kdf.derive(password.encode('utf-8')), salt  # Return both key and salt
    
    def _pad(self, data):
        """
        Pad data to be AES block size (16 bytes).
        """
        padder = padding.PKCS7(128).padder()
        return padder.update(data) + padder.finalize()
    
    def _unpad(self, data):
        """
        Remove padding from decrypted data.
        """
        unpadder = padding.PKCS7(128).unpadder()
        return unpadder.update(data) + unpadder.finalize()
    
    def add_task(self, description, due_date, classification):
        """
        Encrypt and add a task to the encrypted tasks list.
        """
        task = {
            'description': description,
            'due_date': due_date,
            'classification': classification
        }

        # Convert task to string and pad it for encryption
        task_string = f"{task['description']}|{task['due_date']}|{task['classification']}"
        padded_task = self._pad(task_string.encode('utf-8'))

        cipher = Cipher(algorithms.AES(self._encryption_key[0]), modes.CBC(self._encryption_key[1]), backend=default_backend())
        encryptor = cipher.encryptor()
        encrypted_task = encryptor.update(padded_task) + encryptor.finalize()
        self.encrypted_tasks.append(encrypted_task)
    
    def get_tasks(self, view_sensitive=False, otc=None):
        """
        Decrypt and return all tasks.
        """
        tasks = []
        for encrypted_task in self.encrypted_tasks:
            cipher = Cipher(algorithms.AES(self._encryption_key[0]), modes.CBC(self._encryption_key[1]), backend=default_backend())
            decryptor = cipher.decryptor()
            decrypted_task = decryptor.update(encrypted_task) + decryptor.finalize()
            
            # Unpad and decode the decrypted task
            decrypted_task_str = self._unpad(decrypted_task).decode('utf-8')
            description, due_date, classification = decrypted_task_str.split('|')

            if view_sensitive:
                if not Agency.validate_otc(self.agent_id, otc):
                    # Redact sensitive fields if OTC is invalid
                    description = "########"
                    classification = "########"

            tasks.append({
                'description': description,
                'due_date': due_date,
                'classification': classification
            })

        print("Tasks:")
        for idx, task in enumerate(tasks, start=1):
            print(f"{idx}. Due date: {task['due_date']}, Description: {task['description']}, Classification: {task['classification']}")
        
        return tasks
    
    @classmethod
    def register_agent(cls, agent_id, username, password):
        """
        Register a new agent. Ensure the agent ID and username are unique.
        """
        # Check if agent ID or username is already in use
        if agent_id in cls.registered_agents:
            raise ValueError(f"Agent ID '{agent_id}' is already registered.")
        if any(agent.username == username for agent in cls.registered_agents.values()):
            raise ValueError(f"Username '{username}' is already taken.")
        
        # Create and store the new agent
        new_agent = cls(agent_id, username, password)
        cls.registered_agents[agent_id] = new_agent
        return new_agent

    @classmethod
    def get_registered_agents(cls):
        """
        Return a list of registered agents for reference (excluding sensitive info).
        """
        return [repr(agent) for agent in cls.registered_agents.values()]
    
    def __repr__(self):
        return f"User(agent_id={self.agent_id}, username={self.username})"
    
    def save_account_info(self, filepath="accounts.json"):
        """
        Save the user's account information (excluding tasks) to a JSON file.
        Includes improved error handling and ensures thread safety.
        """
        account_data = {
            "agent_id": self.agent_id,
            "username": self.username,
            "hashed_password": self.hashed_password.decode('utf-8'),  # Decode bytes to string
        }

        try:
            # Locking mechanism for thread safety (if implemented elsewhere)
            with open(filepath, "r+") as file:
                try:
                    accounts = json.load(file)
                except json.JSONDecodeError:
                    accounts = {}
        except FileNotFoundError:
            accounts = {}

        # Update the current account info
        accounts[self.agent_id] = account_data

        with open(filepath, "w") as file:
            json.dump(accounts, file, indent=4)

    def save_encrypted_tasks(self, filepath="tasks.dat"):
        """
        Save encrypted tasks to a binary file.
        Incorporates atomic save logic to prevent partial writes.
        """
        temp_filepath = f"{filepath}.tmp"
        with open(temp_filepath, "wb") as file:
            pickle.dump(self.encrypted_tasks, file)
        os.replace(temp_filepath, filepath)  # Atomic replacement

    @classmethod
    def load_account_info(cls, filepath="accounts.json"):
        """
        Load account information from a JSON file and register agents.
        Now handles corrupted JSON data gracefully.
        """
        try:
            with open(filepath, "r") as file:
                try:
                    accounts = json.load(file)
                except json.JSONDecodeError:
                    print("Account file is corrupted. No accounts loaded.")
                    return
            for agent_id, data in accounts.items():
                # Recreate user objects without rehashing passwords
                user = cls.__new__(cls)
                user.agent_id = agent_id
                user.username = data["username"]
                user.hashed_password = data["hashed_password"].encode('utf-8')  # Encode string to bytes
                user._encryption_key = None  # Set key to None; regenerate during login
                user.encrypted_tasks = []
                cls.registered_agents[agent_id] = user
                print(f"Loaded account info: Agent ID: {user.agent_id}, Username: {user.username} Password:{user.hashed_password}")
        except FileNotFoundError:
            print("No account information file found.")

    def load_encrypted_tasks(self, filepath="tasks.dat"):
        """
        Load encrypted tasks from a binary file with enhanced error handling.
        """
        try:
            with open(filepath, "rb") as file:
                self.encrypted_tasks = pickle.load(file)
                # Print the loaded encrypted tasks
                if self.encrypted_tasks:
                    print(f"Loaded {len(self.encrypted_tasks)} encrypted tasks.")
                    for idx, task in enumerate(self.encrypted_tasks, start=1):
                        print(f"Task {idx}: {task}")
                else:
                    print("No tasks found.")
        except FileNotFoundError:
            print("No tasks file found.")
        except pickle.UnpicklingError:
            print("Tasks file is corrupted. Unable to load tasks.")




In [14]:
# Example usage
if __name__ == "__main__":
    # Register a user
    agent = User.register_agent(agent_id="006", username="agent.moustache", password="maths123")
    print(f"Registered agent: {agent}")

    # Agency generates an OTC for the agent
    otc = Agency.generate_otc("006")
    print(f"Generated OTC: {otc}")

    # Validate tasks access with OTC
    # Add tasks properly using the `add_task` method
    agent.add_task(description="Secure the vault.", due_date="2024-12-01", classification="Top Secret")
    agent.add_task(description="Monitor surveillance.", due_date="2024-12-05", classification="Secret")

    print("\nTasks without OTC:")
    print(agent.get_tasks(view_sensitive=True, otc="WRONG_OTC"))  # Invalid OTC

    print("\nTasks with valid OTC:")
    print(agent.get_tasks(view_sensitive=True, otc=otc))  # Valid OTC

    agent.save_account_info()
    agent.save_encrypted_tasks()

    # Load accounts and tasks
    print("\nLoading saved data...")
    User.load_account_info()
    loaded_agent = User.registered_agents["006"]

        # Verify tasks after loading
    print("\nLoaded tasks:")
    loaded_agent.load_encrypted_tasks()

Registered agent: User(agent_id=006, username=agent.moustache)
Generated OTC: ZMTVBR

Tasks without OTC:
Tasks:
1. Due date: 2024-12-01, Description: ########, Classification: ########
2. Due date: 2024-12-05, Description: ########, Classification: ########
[{'description': '########', 'due_date': '2024-12-01', 'classification': '########'}, {'description': '########', 'due_date': '2024-12-05', 'classification': '########'}]

Tasks with valid OTC:
Tasks:
1. Due date: 2024-12-01, Description: Secure the vault., Classification: Top Secret
2. Due date: 2024-12-05, Description: ########, Classification: ########
[{'description': 'Secure the vault.', 'due_date': '2024-12-01', 'classification': 'Top Secret'}, {'description': '########', 'due_date': '2024-12-05', 'classification': '########'}]

Loading saved data...
Loaded account info: Agent ID: 001, Username: agent.smith Password:b'$2b$12$1gS/6/vT7ZB6VTZqTPgQpO.HmeI6y4dqemfEQIrRd0c4XoVYmAOkO'
Loaded account info: Agent ID: 007, Username: ag